# Visão Computacional - Projeto 2-1

### Bruna Kimura

In [2]:
%matplotlib inline

import cv2 as cv2
import numpy as np     
import matplotlib.pyplot as plt

import time

### 1. Detecção de alguns pontos interessantes na imagem

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    # Converte para Grayscale
    bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Parametriza a funcao do OpenCV
    params = dict( maxCorners = 100,
                   qualityLevel = 0.3,
                   minDistance = 7,
                   blockSize = 7 )

    # Funcao que retorna uma lista de pontos
    pts = cv2.goodFeaturesToTrack(bw, mask = None, **params)
    
    # Gera cores de forma aleatória
    color = np.random.randint(0,255,(100,3))

    # Insere uma marcação em cada ponto:
    for i,pt in enumerate(pts):
        a,b = pt.ravel()
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

    cv2.imshow("Video", frame)

captura.release()
cv2.destroyAllWindows()

### 2. Rastreamento dos pontos (apenas em dois frames)

In [ ]:
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

ret, frame = captura.read()
previous = frame
previous_gray = cv2.cvtColor(previous, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

m_x = 0
m_y = 0


while(1):
    ret, frame = captura.read()
    
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.1)
    
    actual = frame
    actual_gray = cv2.cvtColor(actual, cv2.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv2.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv2.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    
    # diferença entre os pontos antigos e os atuais
    dif = np.subtract(good_old, good_new)
    
    #valores correspondentes a x e y
    x = dif[:,0]
    y = dif[:,1]
    
    #quantidade de elementos em x e y
    len_x = len(x)
    len_y = len(y)
    
    soma_x = 0
    soma_y = 0
    
    #soma todos os valores em x e y
    for e in x:
        soma_x += e
        
    for e in y:
        soma_y += e
       
    #divide a soma pela quantidade de elementos, ou seja, encontra a media de x e y 
    m_x += (soma_x/len_x)
    m_y += (soma_y/len_y)
    
    #matriz de translação (utilizando a média de x e y)
    translation_matrix = np.float32([[1,0,int(m_x)], [0,1,int(m_y)]])
    
    img = cv2.add(actual, mask)
    
    #translada a imagem
    num_rows, num_cols = img.shape[:2]
    img_translation = cv2.warpAffine(img, translation_matrix, (num_cols, num_rows))
    
    cv2.imshow('Translation', img_translation)
    
    # Now update the previous frame and previous points
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    
captura.release()
cv2.destroyAllWindows()


### Dense


In [10]:
captura = cv2.VideoCapture(0)
ret, frame1 = captura.read()

prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
prvs_center = prvs[101:361, 201:431]
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

num_rows, num_cols = frame1.shape[:2]

prop = num_cols/num_rows

x_acumulado = 0
y_acumulado = 0

while(1):
    ret, frame2 = captura.read()
    next1 = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    next1_center = next1[101:361, 201:431]
    flow = cv2.calcOpticalFlowFarneback(prvs_center,next1_center, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    x = flow[:,:,0]
    y = flow[:,:,1]
    
    x_m = np.mean(x)
    y_m = np.mean(y)
    
    x_acumulado -= x_m
    y_acumulado -= y_m

    
    translation_matrix = np.float32([[1,0,int(x_acumulado)], [0,1,int(y_acumulado)]])
    
    num_rows, num_cols = frame2.shape[:2]

    img_translation = cv2.warpAffine(frame2, translation_matrix, (num_cols, num_rows))
    
    novo_cols = num_cols - abs(x_acumulado)
    novo_rows = num_rows - abs(y_acumulado)
    
    
    
    novo_prop = novo_cols/novo_rows
    
    if prop > novo_prop:
        y = int(novo_cols * num_rows / num_cols)
        
        if y_acumulado > 0:
            print('p cima')
            if x_acumulado < 0:
                img_cut = img_translation[0:int(novo_cols), int(num_rows-y):num_rows]
            else:
                img_cut = img_translation[int(num_cols-novo_cols):num_cols, int(num_rows-y):num_rows]

        else:
            print('p baixo')
            if x_acumulado < 0:
                img_cut = img_translation[0:int(novo_cols), 0:y]
            else:
                img_cut = img_translation[int(num_cols-novo_cols):num_cols, 0:y]
            
        
    else:
        x = int(novo_rows * num_cols / num_rows)
        
        if y_acumulado > 0:
            print('p cima')
            if x_acumulado < 0:
                img_cut = img_translation[int(num_cols-x):num_cols, int(num_rows-novo_rows):int(num_rows)]
            else:
                img_cut = img_translation[0:x, int(num_rows-novo_rows):int(num_rows)]

        else:
            print('p baixo')
            if x_acumulado < 0:
                img_cut = img_translation[int(num_cols-x):num_cols, 0:int(novo_rows)]
            else:
                img_cut = img_translation[0:x, 0:int(novo_rows)]

#     print(img_cut.shape)
    img_res = cv2.resize(img_cut, (num_cols, num_rows), interpolation = cv2.INTER_CUBIC)
    
    cv2.imshow('Translation', img_translation)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    prvs_center = next1_center
    
captura.release()
cv2.destroyAllWindows()

p baixo
p baixo
p baixo
p baixo
p baixo
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p cima
p cima
p cima
p baixo
p baixo
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p baixo
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima
p cima


In [ ]:
captura.release()